In [25]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
print("Libraries imported.")

Libraries imported.


## Download Dataset 

To obtain the dataset which includes the **postal code, borough and neighborhood** of Toronto, we will utilize the Wikipedia page and by scraping the table in [this link](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), we will be able to generate the required dataframe. 

Get the html of the wikipedia page with **requests** and **beautifulsoup** libraries.

In [7]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

We need 3 lists which includes the postal code, borough and neighborhood.

In [18]:
postalCode=[]
borough=[]
neighborhood=[]

The required information is in the "tr" and "td" HTML items. Former one shows the rows and latter one denotes the ingredient of that row.

In [19]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCode.append(cells[0].text.rstrip('\n'))
        borough.append(cells[1].text.rstrip('\n'))
        neighborhood.append(cells[2].text.rstrip('\n'))

Now we have 3 lists with the information. Next step, we will create dataframe for these.

In [20]:
print(len(postalCode))
print(len(borough))
print(len(neighborhood))

180
180
180


### Create Pandas Dataframe

In [21]:
toronto_df=pd.DataFrame({"PostalCode": postalCode,"Borough": borough,"Neighborhood": neighborhood})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Instruction:** Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [24]:
toronto_df=toronto_df[toronto_df["Borough"] != "Not assigned"].reset_index(drop=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**Instruction**: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [27]:
toronto_df["Neighborhood"]=np.where(toronto_df["Neighborhood"]=="Not assigned",toronto_df["Borough"],toronto_df["Neighborhood"])

### Group by Borough and Postal Codes by putting "," between Neighborhoods. 

In [28]:
toronto_grouped = toronto_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [29]:
toronto_grouped.shape

(103, 3)